In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [392]:
test = pd.read_csv("test1.csv")
df = pd.read_csv("train1_combined.csv")
dev = pd.read_csv("dev1_combined.csv")
#test.drop(test.columns[[13,14]], axis=1, inplace=True)
#df.drop(df.columns[[13,14]], axis=1, inplace=True)

In [393]:
x_train = df.iloc[:,4:-1]
y_train = df.label
x_dev = dev.iloc[:,4:-1]
y_dev = dev.label

In [333]:
df.shape

(28846, 9)

In [334]:
dev.shape

(22036, 9)

In [338]:
len(dev[dev.label ==0])

16036

In [395]:
df.iloc[:,4:]

,aa,ra,jc,pa,dc_A,dc_B,sh_path,label
0,2.508376,0.551129,0.461538,88,0.002317,0.003187,1,1
1,3.049436,0.707973,0.500000,104,0.002317,0.003766,1,1
2,2.984187,0.673785,0.700000,72,0.002317,0.002607,1,1
3,2.027478,0.426129,0.500000,56,0.002317,0.002028,1,1
4,3.060384,0.713467,0.466667,112,0.002317,0.004056,1,1
...,...,...,...,...,...,...,...,...
27353,0.000000,0.000000,0.000000,51,0.002607,0.000290,3,0
27354,0.000000,0.000000,0.000000,27,0.002607,0.000290,999,0
27355,0.000000,0.000000,0.000000,207,0.002607,0.002317,3,0
27356,0.000000,0.000000,0.000000,27,0.002607,0.003476,3,0


In [286]:
test.iloc[:,4:]

,aa,ra,jc,pa
0,0.000000,0.000000,0.000000,56
1,0.000000,0.000000,0.000000,24
2,0.000000,0.000000,0.000000,496
3,1.242670,0.400000,0.080000,72
4,0.000000,0.000000,0.000000,391
...,...,...,...,...
1995,0.000000,0.000000,0.000000,2
1996,0.000000,0.000000,0.000000,2
1997,0.000000,0.000000,0.000000,116
1998,1.766048,0.424368,0.053333,370


In [280]:
y = [True if i < df.shape[0]/2 else False for i in range(df.shape[0])]
x = df.iloc[:,4:13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [281]:
collections.Counter(y)

Counter({True: 16036, False: 16036})

In [423]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
normal = MinMaxScaler()
x_trainN = normal.fit_transform(x_train)
x_devN = normal.transform(x_dev)
#l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'
LR = LogisticRegression(n_jobs = 4).fit(x_trainN, df.label)
LRT = LR.predict(x_devN)

print(metrics.classification_report(dev.label, LRT))


              precision    recall  f1-score   support

           0       0.72      0.97      0.83      5055
           1       0.95      0.63      0.76      5055

    accuracy                           0.80     10110
   macro avg       0.84      0.80      0.79     10110
weighted avg       0.84      0.80      0.79     10110



In [424]:
probs = LR.predict_proba(dev.iloc[:,4:-1])

In [425]:
metrics.roc_auc_score(dev.label, probs[:, 1])

0.6115270892585124

In [415]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:]))

ValueError: X has 4 features per sample; expecting 7

In [352]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.07216161, 0.07062026, 0.07105751, ..., 0.07527991, 1.        ,
       0.07029395])

In [278]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg.csv', header = True, index= False)

### Handmade gridsearch

In [405]:
import itertools as it
from sklearn.metrics import accuracy_score
LR = LogisticRegression
LR_param = {'penalty':['l1', 'l2', 'elasticnet', 'none'],
            'C':[1.0,0.7,0.5,0.3,0.1,0.01], 'max_iter':[200],
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
models = [('Logistic Regression',LR,LR_param)]

result = []
best = {}
for name, m, params in models:
    keys, values = zip(*params.items())
    permutations = [dict(zip(keys, v)) for v in it.product(*values)]
    
    for p in permutations:
        try:
            mod = m(**p)
            mod.fit(x_train, y_train);
            y_pred = mod.predict(x_dev)
            Acc_scored = accuracy_score(y_dev, y_pred)
            cnt = collections.Counter(mod.predict(test.iloc[:,4:]))
            best[str(p)] = (Acc_scored, cnt)
            #print('Model: ',name, ' HyperParams: ',p,' Accuracy: ',Acc_scored)
        except:
            pass#print(p)

    #best_params = max(best, key = lambda k:best[k][0])
    #result.append((name, best_params, best[best_params][0], best[best_params][1]))  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did 

In [406]:
[print(k,v) for k,v in best.items()]

[]

### LGBM

In [382]:
import lightgbm as lgbm

train_data = lgbm.Dataset(x_train, pd.Series(y_train))
test_data = lgbm.Dataset(x_dev, pd.Series(y_dev))

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 4,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=80)

[LightGBM] [Info] Number of positive: 13679, number of negative: 13679
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000087 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 27358, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.800692
Training until validation scores don't improve for 80 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.800649
[3]	valid_0's auc: 0.823213
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.823213
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 0.8

In [383]:
predict

array([0.06371232, 0.05891724, 0.08149031, ..., 0.06371232, 0.99880413,
       0.05097538])

In [384]:
predict= model.predict(test.iloc[:,4:])
collections.Counter(["pos" if i>0.5 else "neg" for i in predict])

Counter({'neg': 1406, 'pos': 594})

### Support Vector Machine (SVM)

In [363]:
SVM = LinearSVC(max_iter=10000)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_dev)


print(metrics.classification_report(y_dev, clf_predict))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.64      0.99      0.78      6000
           1       0.99      0.45      0.62      6000

    accuracy                           0.72     12000
   macro avg       0.81      0.72      0.70     12000
weighted avg       0.81      0.72      0.70     12000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [364]:
probs = clf.predict_proba(x_dev)

In [365]:
probs

array([[0.00746483, 0.99253517],
       [0.00731817, 0.99268183],
       [0.00780515, 0.99219485],
       ...,
       [0.82285244, 0.17714756],
       [0.82191714, 0.17808286],
       [0.82177594, 0.17822406]])

In [366]:
metrics.roc_auc_score(y_dev, probs[:, 1])

0.7539118888888888

In [61]:
collections.Counter(clf.predict(test.iloc[:,4:]))

Counter({'neg': 1465, 'pos': 535})

In [42]:
collections.Counter(y_test)

Counter({'neg': 4029, 'pos': 3989})

In [20]:
results = clf.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.32588784, 0.11201567, 0.32562297, ..., 0.32601893, 0.65587601,
       0.11203549])

In [160]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': predict})
output.to_csv('lightGBM.csv', header = True, index= False)